In [1]:
!pip install modelscope==1.9.5
!pip install transformers==4.39.0
!pip install streamlit==1.24.0
!pip install sentencepiece==0.1.99
!pip install transformers_stream_generator==0.0.4
!pip install datasets==2.18.0
!pip install peft==0.10.0
!pip install openai==1.17.1
!pip install tqdm==4.64.1
!pip install transformers==4.39.3
!python -m pip install setuptools==69.5.1
!pip install vllm==0.4.0.post1
!pip install nest-asyncio
!pip install accelerate
!pip install tf-keras

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/17/d8/f808e32ed7fa86617b9ac7a37b7dcff894c839108c4871cc33ffc4e65b7d/datasets-2.13.0-py3-none-any.whl (485 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.18.0
    Uninstalling datasets-2.18.0:
      Successfully uninstalled datasets-2.18.0

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/a4/73/f620d76193954e16db3d5c53a07d956d7b9c800e570758d3bff91906d4a4/transformers-4.39.0-py3-none-any.whl (8.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3
ERROR: pip's dependency resolver does not currently take into account all the packages th

In [ ]:
import torch
from modelscope import snapshot_download, AutoModel, AutoTokenizer
import os
model_dir = snapshot_download('./qwen/Qwen2-7B-Instruct', cache_dir='./', revision='master')

2024-08-18 17:05:30,871 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-08-18 17:05:30,873 - modelscope - INFO - TensorFlow version 2.17.0 Found.
2024-08-18 17:05:30,873 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-08-18 17:05:30,983 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 3da902487d9387b55bb90991ee9123fc and a total number of 945 components indexed
Downloading: 100%|██████████| 663/663 [00:00<00:00, 758kB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 45.1kB/s]
Downloading: 100%|██████████| 243/243 [00:00<00:00, 269kB/s]
Downloading: 100%|██████████| 11.1k/11.1k [00:00<00:00, 9.11MB/s]
Downloading: 100%|██████████| 1.59M/1.59M [00:00<00:00, 8.52MB/s]
Downloading:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

In [ ]:
model_dir = "./qwen/Qwen2-7B-Instruct"

## 这里运行完请重启notebook

In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [ ]:
import pandas as pd
# 将JSON文件转换为CSV文件
train_data_size = 4000
ana = pd.read_json('data/ana.json')
qwen = pd.read_json('data/qwen_train_data_1697.json')
gsm8k2 = pd.read_json('data/gsm8k2_train_data_7473.json')
#gsm8k2.iloc[:num_records_needed]
temp_df = pd.concat([ana, qwen], ignore_index=True)
num_records_needed = train_data_size - len(temp_df)
gsm8k2 = gsm8k2.iloc[:num_records_needed]
combined_df = pd.concat([temp_df, gsm8k2], ignore_index=True)
shuffled_df = combined_df.sample()
ds = Dataset.from_pandas(combined_df.iloc[:int(train_data_size*0.8)])
dev_ds = Dataset.from_pandas(combined_df.iloc[int(train_data_size*0.8):])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./qwen/Qwen2-7B-Instruct', use_fast=False, trust_remote_code=True)
tokenizer

In [ ]:
def process_func(example):
    MAX_LENGTH = 1800    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你是一个逻辑推理专家，擅长解决逻辑推理问题。<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id
dev_id = dev_ds.map(process_func, remove_columns=ds.column_names)
dev_id

In [ ]:
tokenizer.decode(tokenized_id[0]['input_ids'])

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))


In [ ]:
import torch

model = AutoModelForCausalLM.from_pretrained('./qwen/Qwen2-7B-Instruct', device_map="auto",torch_dtype=torch.bfloat16)
model

In [ ]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
model.dtype

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

In [ ]:
model = get_peft_model(model, config)
config

In [ ]:
model.print_trainable_parameters()

In [ ]:
# args = TrainingArguments(
#     output_dir="./output/Qwen2_instruct_lora",
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=4,
#     logging_steps=10,
#     num_train_epochs=1,
#     save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
#     learning_rate=1e-4,
#     save_on_each_node=True,
#     gradient_checkpointing=True
# )

args = TrainingArguments(
    output_dir="./output/Qwen2_instruct_lora/20240818",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=100,
    eval_steps=100,
    num_train_epochs=5,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    #bf16=True,
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    evaluation_strategy="steps"
    # save_strategy="steps"
)

In [ ]:
#!pip install --upgrade transformers
from transformers import __version__
print(__version__)

In [ ]:
# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=tokenized_id,
#     data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
# )
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    eval_dataset=dev_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
trainer.train()

In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)

In [ ]:
trainer.train()

## 这里运行完请重启notebook

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = './qwen/Qwen2-7B-Instruct/'
lora_path = './output/Qwen2_instruct_lora_an/checkpoint-100' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.float16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = '''你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：\n\n### 题目:\n假设您需要构建一个二叉搜索树，其中每个节点或者是一个空的节点（称为"空节点"），或者是一个包含一个整数值和两个子树的节点（称为"数值节点"）。以下是构建这棵树的规则：\n\n1. 树中不存在重复的元素。\n2. 对于每个数值节点，其左子树的所有值都小于该节点的值，其右子树的所有值都大于该节点的值。\n3. 插入一个新值到一个"空节点"时，该"空节点"会被一个包含新值的新的数值节点取代。\n4. 插入一个已存在的数值将不会改变树。\n\n请基于以上规则，回答以下选择题：\n\n### 问题:\n选择题 1：\n给定一个空的二叉搜索树，插入下列数字: [5, 9, 2, 10, 11, 3]，下面哪个选项正确描述了结果树的结构？\nA. tree(5, tree(2, tree(3, nil, nil), nil), tree(9, tree(10, nil, nil), tree(11, nil, nil)))\nB. tree(5, tree(2, nil, tree(3, nil, nil)), tree(9, nil, tree(10, nil, tree(11, nil, nil))))\nC. tree(5, tree(3, tree(2, nil, nil), nil), tree(9, nil, tree(10, tree(11, nil, nil), nil)))\nD. tree(5, nil, tree(2, nil, tree(3, nil, nil)), tree(9, tree(11, nil, nil), tree(10, nil, nil)))'''
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "你是一个逻辑推理专家，擅长解决逻辑推理问题。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# 模型合并存储

new_model_directory = "./merged_model_an"
merged_model = model.merge_and_unload()
# 将权重保存为safetensors格式的权重, 且每个权重文件最大不超过2GB(2048MB)
merged_model.save_pretrained(new_model_directory, max_shard_size="2048MB", safe_serialization=True)


In [ ]:
!cp ./qwen/Qwen2-7B-Instruct/tokenizer.json ./merged_model_an/

## 这里运行完请重启notebook